# Pathway A

## Overall units:
   energy: GWh
   capacity: GW
   area: km^2
   money: €

# Assumptions from task description

In [1]:
# Define customers
customers = ['Customer_1_Steel_Plant', 'Customer_2_Chemical_Plant', 'Customer_3_Airport']

# Define time horizon
time_horizon = range(1, 11)  # Time periods from 1 to 10

# Define selling prices
price_per_unit = {
    'hydrogen': 210000,
    'ammonia': 287004,
    'jetfuel': 315000,
}

# Define typical capacity per unit area in GW/km^2
capacity_per_unit = {
    'photovoltaic': 20,
    'wind': 30,
}

# Define the CO2 point source availabilty in tCO2 /a
m = - 150000 / (2050-2023)
b = 150000 

point_source_availability = {}
for t in time_horizon:
    point_source_availability[t-1] = m*(t-1) + b
    if t > 27:
        point_source_availability[t-1] = 0

# Our own assumptions

In [2]:
# Decision variables
x = {
    'photovoltaic': 1,
    'wind': 0,
    'PEM_electrolyzer': 1,
    'alkaline_electrolyzer': 0,
    'FT_synthesis': 0,
    'ammonia_synthesis': 0,
}

# Define supply to customers
y = {}
y['Customer_1_Steel_Plant'] = {}
y['Customer_2_Chemical_Plant'] = {}
y['Customer_3_Airport'] = {}

for t in time_horizon:
    y['Customer_1_Steel_Plant'][t-1] = 1000
    y['Customer_2_Chemical_Plant'][t-1] = 1000
    y['Customer_3_Airport'][t-1] = 1000

# Discount rate
i = 0.1

print(y)

{'Customer_1_Steel_Plant': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 'Customer_2_Chemical_Plant': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 'Customer_3_Airport': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}}


# Let's start the NPV calculation

In [3]:
# Decision variables
x = {
    'photovoltaic': 1,
    'wind': 0,
    'PEM_electrolyzer': 1,
    'alkaline_electrolyzer': 0,
    'FT_synthesis': 0,
    'ammonia_synthesis': 0,
}

x_transport = {
    'hydrogen': 1,
    'ammonia': 0,
    'jetfuel': 0,
}

# Define supply to customers in GWh
y = {}
y['Customer_1_Steel_Plant'] = {}
y['Customer_2_Chemical_Plant'] = {}
y['Customer_3_Airport'] = {}

for t in time_horizon:
    y['Customer_1_Steel_Plant'][t-1] = 1000
    y['Customer_2_Chemical_Plant'][t-1] = 1000
    y['Customer_3_Airport'][t-1] = 1000

# Define the size of technologies in GW
capacity = {
    'photovoltaic': 1,
    'wind': 0,
    'PEM_electrolyzer': 1,
    'alkaline_electrolyzer': 0,
    'FT_synthesis': 1,
    'ammonia_synthesis': 1,
}

# Define transportedd amounts in GWh
transported_product = {
    'ammonia':  [y['Customer_2_Chemical_Plant'][t-1] * x_transport['ammonia'] for t in time_horizon],
    'jetfuel': [y['Customer_3_Airport'][t-1] * x_transport['jetfuel'] for t in time_horizon],
    'hydrogen': [y['Customer_1_Steel_Plant'][t-1] * x_transport['hydrogen'] + y['Customer_2_Chemical_Plant'][t-1] * (1- x_transport['ammonia']) / 0.8 + y['Customer_3_Airport'][t-1] * (1- x_transport['jetfuel']) / (0.71 * 0.75)  for t in time_horizon]
}

print(y)

technologies = ['photovoltaic', 'wind', 'PEM_electrolyzer', 'alkaline_electrolyzer', 'FT_synthesis', 'ammonia_synthesis']
products = ['hydrogen', 'ammonia', 'jetfuel']

capex = {
    'photovoltaic': 650 * 10 ** 6,
    'wind': 1500 * 10 ** 6,
    'PEM_electrolyzer': 700 * 10 ** 6,
    'alkaline_electrolyzer': 650 * 10 ** 6,
    'FT_synthesis': 1200 * 10 ** 6,
    'ammonia_synthesis': 1400 * 10 ** 6
}

opex = {
    'photovoltaic': 0.03 * capex['photovoltaic'],
    'wind': 0.04 * capex['wind'],
    'PEM_electrolyzer': 0.04 * capex['PEM_electrolyzer'],
    'alkaline_electrolyzer': 0.04 * capex['alkaline_electrolyzer'],
    'FT_synthesis': 0.05 * capex['FT_synthesis'],
    'ammonia_synthesis': 0.05 * capex['ammonia_synthesis'],
}

transport_costs = {
    'hydrogen': 20 * 10**3, 
    'ammonia': 10 * 10**3,
    'jetfuel': 5 * 10**3,
}

cash_inflow_customer = {}

for c in customers:
    cash_inflow_customer[c] = {}
    for t in time_horizon:
        cash_inflow_customer[c][t-1] = price_per_unit['hydrogen'] * y[c][t-1]

print(cash_inflow_customer)

# Initial Investment
init_investment = sum(capex[i] * x[i] for i in technologies)
print(init_investment)

cash_inflow = {}
for t in time_horizon:
    cash_inflow[t-1] = sum(cash_inflow_customer[c][t-1] for c in customers for t in time_horizon)

print(cash_inflow)

cash_outflow_technology = {}

for i in technologies:
    cash_outflow_technology[i] = {}
    for t in time_horizon:
        cash_outflow_technology[i][t-1] = opex[i]  * capacity[i]

print(cash_outflow_technology)

cash_outflow_transport = {}
for i in products:
    cash_outflow_transport[i] = {}
    for t in time_horizon:
        cash_outflow_transport[i][t-1] = transported_product[i][t-1] * transport_costs[i]

print(cash_outflow_transport)

cash_outflow = {}
for t in time_horizon:
    cash_outflow[t-1] = sum(cash_outflow_technology[i][t-1] for i in technologies) + sum(cash_outflow_transport[i][t-1] for i in products)

{'Customer_1_Steel_Plant': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 'Customer_2_Chemical_Plant': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}, 'Customer_3_Airport': {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}}
{'Customer_1_Steel_Plant': {0: 210000000, 1: 210000000, 2: 210000000, 3: 210000000, 4: 210000000, 5: 210000000, 6: 210000000, 7: 210000000, 8: 210000000, 9: 210000000}, 'Customer_2_Chemical_Plant': {0: 210000000, 1: 210000000, 2: 210000000, 3: 210000000, 4: 210000000, 5: 210000000, 6: 210000000, 7: 210000000, 8: 210000000, 9: 210000000}, 'Customer_3_Airport': {0: 210000000, 1: 210000000, 2: 210000000, 3: 210000000, 4: 210000000, 5: 210000000, 6: 210000000, 7: 210000000, 8: 210000000, 9: 210000000}}
1350000000
{0: 6300000000, 1: 6300000000, 2: 6300000000, 3: 6300000000, 4: 6300000000, 5: 6300000000, 6: 6300000000, 7: 6300000000, 8: 6300000

In [4]:
i = 0.1

NPV = - init_investment + sum(
    ( cash_inflow[t - 1]
    - cash_outflow[t - 1]
    ) 
    / ((1 + i) ** t)
    for t in time_horizon
)

print(NPV)

35762824721.79513
